In [1]:
# Import necessary packagess
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import torch

In [2]:
# Load csv to pandas dataframes
train = pd.read_csv("./train_2kmZucJ.csv")
test = pd.read_csv("./test_oJQbWVk.csv")

In [3]:
# Quick check on the data
train.head(20)

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...
5,6,1,iPhone software update fucked up my phone big ...
6,7,0,Happy for us .. #instapic #instadaily #us #son...
7,8,0,New Type C charger cable #UK http://www.ebay.c...
8,9,0,Bout to go shopping again listening to music #...
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...


In [4]:
# Function to clean tweets
def clean_tweets(tweet):

    # Change all text to lowercase
    tweet = tweet.lower()

    def remove_patterns(string, pattern):
        matches = re.findall(pattern, string)
        for match in matches: 
            string = re.sub(match, "", string)
        return string
    
    # Removing user handles
    if isinstance(tweet, str):
        tweet = remove_patterns(tweet, r"@[\w]*")
    elif isinstance(tweet, bytes):
        tweet = remove_patterns(tweet.decode("utf-8"), r"@[\w]*")

    # Removing URLs
    tweet = re.sub(r'http?://\S+|www\.\S+', " ", tweet)
    tweet = re.sub(r'https?://\S+|www\.\S+', " ", tweet)
    tweet = re.sub(r'pic.\S+', " ", tweet)

    # Removing special characters
    tweet = re.sub(r"[^a-zA-Z#']", " ", tweet)

    # Removing one-letter words
    tweet = ' '.join([word for word in tweet.split() if len(word) > 1])

    # Removing trailing/leading whitespaces
    tweet = tweet.strip()

    return tweet

train['cleaned_tweet']=train['tweet'].apply(clean_tweets)
test['cleaned_tweet']=test['tweet'].apply(clean_tweets)
    

In [5]:
train.head()

,id,label,tweet,cleaned_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,#fingerprint #pregnancy test #android #apps #b...
1,2,0,Finally a transparant silicon case ^^ Thanks t...,finally transparant silicon case thanks to my ...
2,3,0,We love this! Would you go? #talk #makememorie...,we love this would you go #talk #makememories ...
3,4,0,I'm wired I know I'm George I was made that wa...,i'm wired know i'm george was made that way #i...
4,5,1,What amazing service! Apple won't even talk to...,what amazing service apple won't even talk to ...


In [6]:
from datasets import Dataset

# Convert pandas dataframes into Datasets
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [7]:
# Check the dataset
train_dataset

Dataset({
    features: ['id', 'tweet', 'cleaned_tweet'],
    num_rows: 1953
})

In [8]:
# Tokenizing the tweets
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [18]:
def tokenize_function(examples):
    return tokenizer(examples['cleaned_tweet'], padding="max_length", truncation=True, max_length=128)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7920 [00:00<?, ? examples/s]

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

In [19]:
# Move process to GPU if cuda is available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [21]:
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["id", "tweet", "cleaned_tweet"])
tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
tokenized_train_dataset.set_format("torch")

tokenized_test_dataset = tokenized_test_dataset.remove_columns(["id", "tweet", "cleaned_tweet"])
tokenized_test_dataset.set_format("torch")

In [22]:
from transformers import AutoModelForSequenceClassification

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.we

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [24]:
from transformers import TrainingArguments, Trainer

# Arguments for the training model (modify accordingly to hardware)
training_args = TrainingArguments(
   output_dir='./results',
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   gradient_accumulation_steps=4,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
)

from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

trainer = Trainer(
   model = model,
   args = training_args,
   train_dataset = tokenized_train_dataset,
   eval_dataset = tokenized_test_dataset,
   tokenizer = tokenizer,
   compute_metrics = compute_metrics)

In [25]:
# Start the model training
trainer.train()

C:\Users\Johanes Steven\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss


TrainOutput(global_step=246, training_loss=0.23485557432097148, metrics={'train_runtime': 2703.4387, 'train_samples_per_second': 5.859, 'train_steps_per_second': 0.091, 'total_flos': 521391681110016.0, 'train_loss': 0.23485557432097148, 'epoch': 1.99})

In [26]:
# Prepare the test dataset
test_dataloader = torch.utils.data.DataLoader(tokenized_test_dataset, batch_size=16)

In [27]:
model.eval()
predictions = []

In [28]:
with torch.no_grad():
    for batch in test_dataloader:
        # Move the batch to the GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# Combine predictions with ids
predictions_df = pd.DataFrame({'id': test['id'], 'label': predictions})

# Write predictions to file
predictions_df.to_csv("predictions.csv", index=False)